In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import os

In [15]:
# Crie uma sessão Spark
spark = SparkSession.builder \
    .appName("tech-challenge-fase3") \
    .getOrCreate()

In [16]:
# Função para obter o próximo número de tabela
def get_next_table_number(parquet_dir):
    # Lista todos os arquivos no diretório
    files = os.listdir(parquet_dir)
    
    # Filtra os arquivos que correspondem ao padrão "tbx{number}_{table_name}.parquet"
    table_numbers = [
        int(f.split('_')[0][3:]) for f in files if f.startswith('tbx') and f.endswith('.parquet')
    ]
    
    # Determina o próximo número de tabela
    if table_numbers:
        next_table_number = max(table_numbers) + 1
    else:
        next_table_number = 1
    
    # Retorna o número da tabela como string completando com zeros até 3 casas
    return str(next_table_number).zfill(3)

In [17]:
# Função para salvar DataFrame em formato Parquet
def save_to_parquet(df, table_name):
    # Diretório onde os arquivos Parquet são salvos
    parquet_dir = "./data/parquet/"
    
    # Verifica se já existe um arquivo com o mesmo table_name
    existing_files = [f for f in os.listdir(parquet_dir) if f.endswith(f"_{table_name}.parquet")]
    
    if existing_files:
        # Extrai o número da tabela do arquivo existente
        existing_file = existing_files[0]
        table_number = existing_file.split('_')[0][3:]
    else:         
        # Obtém o próximo número de tabela
        table_number = get_next_table_number(parquet_dir)
    
    # Salva o DataFrame em formato Parquet
    df.write \
        .format("parquet") \
        .mode("overwrite") \
        .save(f"{parquet_dir}tbx{table_number}_{table_name}.parquet")

In [18]:
df = spark\
    .read\
    .option('delimiter',',')\
    .option('header',True)\
    .option('inferSchema',True)\
    .csv('./data/raw')

columns = [
    "UF", "V1012", "V1013", "V1022", "A002", "A003",
    "A004", "A005", "B0011", "B0014", "B0015", "B0019",
    "B00111", "B002", "B0031", "B005", "B007", "C007B",
    "C01011", "F001",'B009B'
]

df = df.select(columns)

# Renomeie as colunas
df = df\
        .withColumnRenamed("UF", "uf")\
        .withColumnRenamed("V1012", "semana_mes")\
        .withColumnRenamed("V1013", "mes")\
        .withColumnRenamed("V1022", "area_domicilio")\
        .withColumnRenamed("A002", "idade")\
        .withColumnRenamed("A003", "sexo")\
        .withColumnRenamed("A004", "cor_raca")\
        .withColumnRenamed("A005", "escolaridade")\
        .withColumnRenamed("B0011", "teve_febre")\
        .withColumnRenamed("B0014", "teve_dificuldade_respirar")\
        .withColumnRenamed("B0015", "teve_dor_cabeca")\
        .withColumnRenamed("B0019", "teve_fadiga")\
        .withColumnRenamed("B00111", "teve_perda_cheiro")\
        .withColumnRenamed("B002", "foi_posto_saude")\
        .withColumnRenamed("B0031", "ficou_em_casa")\
        .withColumnRenamed("B005", "ficou_internado")\
        .withColumnRenamed("B009B", "resultado_covid")\
        .withColumnRenamed("B007", "tem_plano_saude")\
        .withColumnRenamed("C007B", "assalariado")\
        .withColumnRenamed("C01011", "faixa_rendimento")\
        .withColumnRenamed("F001", "situacao_domicilio")


# Salve o DataFrame em formato Parquet
save_to_parquet(df, "data")

24/08/18 18:01:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: UF, V1012, V1013, V1022, A002, A003, A004, A005, A007A, B0011, B0012, B0016, B0018, B00111, B00112, B0044, B0046, B007, C006, C009A, E0022
 Schema: UF, V1012, V1013, V1022, A002, A003, A004, A005, B0011, B0014, B0015, B0019, B00111, B002, B0031, B005, B007, B009B, C007B, C01011, F001
Expected: B0011 but found: A007A
CSV file: file:///home/spark/source/repos/fiap-dataanalysis-techchallenge-fase3/data/raw/PNAD_COVID_112020.csv
24/08/18 18:01:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: UF, V1012, V1013, V1022, A002, A003, A004, A005, B0015, B0018, B0019, B002, B0032, B0035, B0036, C003, C005, C0053, C011A11, D0013, 
 Schema: UF, V1012, V1013, V1022, A002, A003, A004, A005, B0011, B0014, B0015, B0019, B00111, B002, B0031, B005, B007, B009B, C007B, C01011, F001
Expected: B0011 but found: B0015
CSV file: file:///home/spark/source/repos/fiap-dataanalysis-techchallenge

In [19]:
# Define o esquema para o DataFrame
schema = T.StructType(
    [
        T.StructField('id', T.StringType()),  # Código
        T.StructField('name', T.StringType())   # Nome
    ]
)

In [20]:
# Dados dos estados brasileiros com seus respectivos códigos
data = [
    {
        "11": "Rondônia",
        "12": "Acre",
        "13": "Amazonas",
        "14": "Roraima",
        "15": "Pará",
        "16": "Amapá",
        "17": "Tocantins",
        "21": "Maranhão",
        "22": "Piauí",
        "23": "Ceará",
        "24": "Rio Grande do Norte",
        "25": "Paraíba",
        "26": "Pernambuco",
        "27": "Alagoas",
        "28": "Sergipe",
        "29": "Bahia",
        "31": "Minas Gerais",
        "32": "Espírito Santo",
        "33": "Rio de Janeiro",
        "35": "São Paulo",
        "41": "Paraná",
        "42": "Santa Catarina",
        "43": "Rio Grande do Sul",
        "50": "Mato Grosso do Sul",
        "51": "Mato Grosso",
        "52": "Goiás",
        "53": "Distrito Federal"
    }
]
# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "states")

In [21]:
# Dados de sexo com seus respectivos códigos
data = [
    {
        '1': 'Masculino',
        '2': 'Feminino'
    }
]
# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

df_data.show()

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data,"gender")

+---+---------+
| id|     name|
+---+---------+
|  1|Masculino|
|  2| Feminino|
+---+---------+



In [22]:
# Dados das áreas de domicílio com seus respectivos códigos
data = [
    {
        '1': 'Urbana',
        '2': 'Rural'
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "residential_area")

In [23]:
# Dados das raças com seus respectivos códigos
data = [
    {
        '1': 'Branca',
        '2': 'Preta',
        '3': 'Amarela',
        '4': 'Parda',
        '5': 'Indígena',
        '9': 'Ignorado'
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "race")

In [24]:
# Dados de escolaridade com seus respectivos códigos
data = [
    {
        '1': 'Sem instrução',
        '2': 'Fundamental incompleto',
        '3': 'Fundamental completa',
        '4': 'Médio incompleto',
        '5': 'Médio completo',
        '6': 'Superior incompleto',
        '7': 'Superior completo',
        '8': 'Pós-graduação, mestrado ou doutorado'
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "education_levels")

In [25]:
# Dados das respostas de COVID com seus respectivos códigos
data = [
    {
        '1': 'Sim',
        '2': 'Não ',
        '3': 'Não sabe',
        '9': 'Ignorado'
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "covid_responses")

In [26]:
# Dados das respostas de internação com seus respectivos códigos
data = [
    {
        '1': 'Sim',
        '2': 'Não ',
        '3': 'Não foi atendido',
        '9': 'Ignorado'
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "hospitalization_status")


In [27]:
# Dados das faixas de rendimento com seus respectivos códigos
data = [
    {
        '00':   '0 - 100',
        '01':	'101 - 300',
        '02':	'301 - 600',
        '03':	'601 - 800',
        '04':	'801 - 1.600',
        '05':	'1.601 - 3.000',
        '06':	'3.001 - 10.000',
        '07':	'10.001 - 50.000',
        '08':	'50.001 - 100.000',
        '09':	'Mais de 100.000'
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)
# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "income_brackets")


In [28]:
# Dados das situações de domicílio com seus respectivos códigos
data = [
    {
        '1': 'Próprio - já pago ',
        '2': 'Próprio - ainda pagando',
        '3': 'Alugado',
        '4': 'Cedido por empregador',
        '5': 'Cedido por familiar ',
        '6': 'Cedido de outra forma ',
        '7': 'Outra condição',
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "residential_situations")

In [29]:
# Dados das situações de domicílio com seus respectivos códigos
data = [
    {
        "UF": "uf",
        "V1012": "semana_mes",
        "V1013": "mes",
        "V1022": "area_domicilio",
        "A002": "idade",
        "A003": "sexo",
        "A004": "cor_raca",
        "A005": "escolaridade",
        "B0011": "teve_febre",
        "B0014": "teve_dificuldade_respirar",
        "B0015": "teve_dor_cabeca",
        "B0019": "teve_fadiga",
        "B00111": "teve_perda_cheiro",
        "B002": "foi_posto_saude",
        "B0031": "ficou_em_casa",
        "B005": "ficou_internado",
        "B007": "tem_plano_saude",
        "C007B": "assalariado",
        "C01011": "faixa_rendimento",
        "F001": "situacao_domicilio"
    }
]

# Transformar o array de dicionários em uma lista de tuplas
transformed_data = [(k, v) for d in data for k, v in d.items()]

# Cria um DataFrame com os dados de sexo
df_data = spark.createDataFrame(data=transformed_data, schema=schema)

# Salva o DataFrame em formato Parquet
save_to_parquet(df_data, "questions")